# Proyecto de Inteligencia Artificial  
## Clasificación de riesgo crediticio (`Credit Score`)

En este proyecto se analiza un conjunto de datos de clientes con información demográfica y financiera, con el objetivo de **predecir el nivel de riesgo crediticio** de cada persona.

El flujo general será:

1. Exploración y descripción del dataset.
2. Preprocesamiento de datos (limpieza, tratamiento de nulos, codificación, escalado).
3. Entrenamiento y evaluación de modelos de clasificación.
4. Reducción de dimensionalidad con PCA.
5. Clustering (aprendizaje no supervisado).
6. Análisis y conclusiones finales.


In [3]:
import pandas as pd
pd.set_option("display.max_columns", 50)

df = pd.read_csv("credit_score.csv")

# limpiar posibles espacios en nombres de columnas (por si acaso)
df.columns = df.columns.str.strip()

df.head()


C:\Users\COMPU\AppData\Local\Temp\ipykernel_8096\777321449.py:4: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("credit_score.csv")


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,_,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4,6.27,4.0,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",6,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [4]:
# Tamaño del dataset (filas, columnas)
df.shape

(100000, 28)

In [5]:
# Lista de columnas
df.columns.tolist()

['ID',
 'Customer_ID',
 'Month',
 'Name',
 'Age',
 'SSN',
 'Occupation',
 'Annual_Income',
 'Monthly_Inhand_Salary',
 'Num_Bank_Accounts',
 'Num_Credit_Card',
 'Interest_Rate',
 'Num_of_Loan',
 'Type_of_Loan',
 'Delay_from_due_date',
 'Num_of_Delayed_Payment',
 'Changed_Credit_Limit',
 'Num_Credit_Inquiries',
 'Credit_Mix',
 'Outstanding_Debt',
 'Credit_Utilization_Ratio',
 'Credit_History_Age',
 'Payment_of_Min_Amount',
 'Total_EMI_per_month',
 'Amount_invested_monthly',
 'Payment_Behaviour',
 'Monthly_Balance',
 'Credit_Score']

In [6]:
# Información general del DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [13]:
#resumen estadístico:
df.describe(include="all")

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
count,100000,100000,100000,90015,100000,100000,100000,100000,84998.000000,100000.000000,100000.00000,100000.000000,100000,88592,100000.000000,92998,100000,98035.000000,100000,100000,100000.000000,90970,100000,100000.000000,95521,100000,98800,100000
unique,100000,12500,8,10139,1788,12501,16,18940,NaN,NaN,NaN,NaN,434,6260,NaN,749,4384,NaN,4,13178,NaN,404,3,NaN,91049,7,98792,3
top,0x1602,CUS_0xd40,January,Langep,38,#F%$D@*&8,_______,36585.12,NaN,NaN,NaN,NaN,3,Not Specified,NaN,19,_,NaN,Standard,1360.45,NaN,15 Years and 11 Months,Yes,NaN,__10000__,Low_spent_Small_value_payments,__-333333333333333333333333333__,Standard
freq,1,8,12500,44,2833,5572,7062,16,NaN,NaN,NaN,NaN,14386,1408,NaN,5327,2091,NaN,36479,24,NaN,446,52326,NaN,4305,25513,9,53174
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4194.170850,17.091280,22.47443,72.466040,NaN,NaN,21.068780,NaN,NaN,27.754251,NaN,NaN,32.285173,NaN,NaN,1403.118217,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3183.686167,117.404834,129.05741,466.422621,NaN,NaN,14.860104,NaN,NaN,193.177339,NaN,NaN,5.116875,NaN,NaN,8306.041270,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,303.645417,-1.000000,0.00000,1.000000,NaN,NaN,-5.000000,NaN,NaN,0.000000,NaN,NaN,20.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1625.568229,3.000000,4.00000,8.000000,NaN,NaN,10.000000,NaN,NaN,3.000000,NaN,NaN,28.052567,NaN,NaN,30.306660,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3093.745000,6.000000,5.00000,13.000000,NaN,NaN,18.000000,NaN,NaN,6.000000,NaN,NaN,32.305784,NaN,NaN,69.249473,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5957.448333,7.000000,7.00000,20.000000,NaN,NaN,28.000000,NaN,NaN,9.000000,NaN,NaN,36.496663,NaN,NaN,161.224249,NaN,NaN,NaN,NaN


In [8]:
#distribución de la variable objetivo
df["Credit_Score"].value_counts()

Credit_Score
Standard    53174
Poor        28998
Good        17828
Name: count, dtype: int64

In [14]:
#porcentajes
df["Credit_Score"].value_counts(normalize=True)

Credit_Score
Standard    0.53174
Poor        0.28998
Good        0.17828
Name: proportion, dtype: float64

# 1. Descripción detallada del dataset

## 1.1 Origen y fuente

El conjunto de datos utilizado proviene de la plataforma **Kaggle**, del problema de clasificación de puntaje crediticio (“Credit Score Classification”).  
El archivo que se utiliza en este proyecto es **`train.csv`**, el cual contiene la información crediticia y financiera de distintos clientes de una entidad financiera.

- Número de instancias (filas): `N_filas`  
- Número de atributos (columnas): `N_columnas`

La variable objetivo del problema es **`Credit_Score`**, que clasifica a cada cliente en tres categorías de riesgo crediticio:

- `Poor`
- `Standard`
- `Good`

Cada fila representa el estado de un cliente en un determinado mes (información bancaria + comportamiento de pago).

## 1.2 Propósito del dataset

El dataset está orientado a un problema de **clasificación supervisada**:  
> Dada la información financiera y de comportamiento de pago de una persona, predecir su categoría de **puntaje crediticio** (`Credit_Score`).

Este propósito está alineado con el objetivo del proyecto de IA: construir y evaluar modelos de *machine learning* que automaticen la clasificación del riesgo crediticio.

### 1.3 Significado general de las variables

Las variables del dataset representan distintos aspectos de la situación financiera y del comportamiento de pago de cada cliente. A nivel conceptual se pueden agrupar así:

- **Datos demográficos**
  - `Age`: edad del cliente (años).  
  - `Occupation`: tipo de ocupación o profesión (Scientist, Teacher, Engineer, Lawyer, etc.).  
  Estas variables describen quién es la persona y se relacionan indirectamente con estabilidad laboral e historial crediticio.

- **Ingresos y relación con el sistema financiero**
  - `Annual_Income`: ingreso anual aproximado del cliente.  
  - `Monthly_Inhand_Salary`: salario mensual neto (lo que recibe en mano).  
  - `Num_Bank_Accounts`: número de cuentas bancarias que posee.  
  - `Num_Credit_Card`: número de tarjetas de crédito.  
  Estas variables miden cuánto gana la persona y qué tan vinculada está al sistema bancario.

- **Nivel de endeudamiento y uso del crédito**
  - `Num_of_Loan`: número de préstamos activos.  
  - `Type_of_Loan`: tipos de préstamos que tiene (auto, personal, educativo, etc.).  
  - `Outstanding_Debt`: deuda pendiente total (en USD).  
  - `Credit_Utilization_Ratio`: porcentaje del límite de crédito que está siendo utilizado.  
  - `Total_EMI_per_month`: suma de todas las cuotas mensuales de préstamos.  
  - `Amount_invested_monthly`: monto invertido mensualmente.  
  - `Monthly_Balance`: saldo mensual aproximado después de pagar gastos y deudas.  
  En conjunto, estas variables indican cuánto debe el cliente, cuánto paga cada mes y qué tanto margen le queda para ahorrar o invertir.

- **Historial y comportamiento de pago**
  - `Delay_from_due_date`: días promedio de retraso respecto a la fecha de vencimiento.  
  - `Num_of_Delayed_Payment`: número de pagos realizados con retraso.  
  - `Changed_Credit_Limit`: cambio porcentual en el límite de crédito.  
  - `Num_Credit_Inquiries`: número de consultas de crédito recientes.  
  - `Credit_History_Age`: antigüedad del historial crediticio (años y meses).  
  - `Payment_of_Min_Amount`: indica si suele pagar solo el monto mínimo (Yes/No/NM).  
  - `Payment_Behaviour`: patrón de gasto y pago (por ejemplo, Low_spent_Small_value_payments, High_spent_Medium_value_payments, etc.).  
  - `Credit_Mix`: calidad de la mezcla de créditos (Good, Standard, Bad, `_`).  
  Estas variables reflejan cómo se comporta el cliente con sus deudas: si paga a tiempo, si se atrasa, si pide mucho crédito, si su límite sube o baja, etc.

- **Identificadores y variable objetivo**
  - `ID`, `Customer_ID`, `Name`, `SSN`, `Month`: identifican al registro, al cliente y al mes; no aportan información directa sobre el riesgo, por lo que se usan solo para referencia.  
  - `Credit_Score`: etiqueta de puntaje crediticio del cliente con tres categorías (`Poor`, `Standard`, `Good`); es la **variable objetivo** que el modelo de Machine Learning debe predecir.

---

### 1.4 Calidad de los datos: valores faltantes y valores atípicos

El dataset presenta varios problemas de calidad de datos que deben ser tratados en el preprocesamiento:

- **Valores faltantes (NaN)**  
  Algunas columnas tienen una proporción importante de valores faltantes:
  - `Monthly_Inhand_Salary`: 15,00 % de valores faltantes.  
  - `Type_of_Loan`: 11,41 %.  
  - `Name`: 9,98 %.  
  - `Credit_History_Age`: 9,03 %.  
  - `Num_of_Delayed_Payment`: 7,00 %.  
  - `Amount_invested_monthly`: 4,48 %.  
  - `Num_Credit_Inquiries`: 1,96 %.  
  - `Monthly_Balance`: 1,20 %.  
  El resto de variables no presenta valores faltantes.  
  Estos huecos obligan a definir estrategias de imputación (por ejemplo, rellenar numéricas con la mediana y categóricas con la moda).

- **Valores “basura” o codificaciones extrañas**  
  Además de los NaN explícitos, existen valores que representan información perdida o mal registrada, como:
  - En `Age`: valores como `-500`, `7580`, `28_`, `34_`, que no son edades reales.  
  - En `Amount_invested_monthly`: el valor especial `__10000__`, que se considera un código anómalo.  
  - En `Payment_Behaviour`: la categoría `!@9#%8`, que no corresponde a ningún patrón de pago válido.  
  - En `Credit_Mix`: el valor `_`, usado como desconocido.  
  Estos valores deben tratarse como datos faltantes o corregirse para evitar que distorsionen el entrenamiento del modelo.

- **Outliers y rangos no realistas**  
  Algunas variables numéricas presentan rangos poco realistas, por ejemplo:
  - `Age` contiene edades negativas o extremadamente altas (por encima de 8000), que claramente no representan la edad real de una persona.  
  - También pueden existir montos de deuda o inversión muy elevados respecto al resto de la distribución.  
  Estos casos se consideran outliers y requieren tratamiento (por ejemplo, recorte de rangos, transformaciones o revisión manual).

---

### 1.5 Desbalance de la variable objetivo (`Credit_Score`)

La variable objetivo `Credit_Score` no está distribuida de forma uniforme. En el dataset de entrenamiento se observa:

- `Standard`: 53,17 % de los registros (53 174 filas).  
- `Poor`: 29,00 % (28 998 filas).  
- `Good`: 17,83 % (17 828 filas).

Esto significa que el problema de clasificación es **desbalanceado**: la clase `Standard` es claramente mayoritaria, mientras que la clase `Good` es minoritaria.  
Este desbalance puede hacer que algunos modelos tiendan a favorecer la clase dominante, por lo que más adelante será necesario analizar métricas por clase (precision, recall, F1-score) e incluso considerar técnicas de balanceo de datos si fuera necesario.


